# Detecting Payment Card Fraud

In this section, we'll look at a credit card fraud detection dataset, and build a binary classification model that can identify transactions as either fraudulent or valid, based on provided, *historical* data. In a [2016 study](https://nilsonreport.com/upload/content_promo/The_Nilson_Report_10-17-2016.pdf), it was estimated that credit card fraud was responsible for over 20 billion dollars in loss, worldwide. Accurately detecting cases of fraud is an ongoing area of research.

<img src=notebook_ims/fraud_detection.png width=50% />

### Labeled Data

The payment fraud data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data). This has features and labels for thousands of credit card transactions, each of which is labeled as fraudulent or valid. In this notebook, we'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.

### Binary Classification

Since we have true labels to aim for, we'll take a **supervised learning** approach and train a binary classifier to sort data into one of our two transaction classes: fraudulent or valid.  We'll train a model on training data and see how well it generalizes on some test data.

The notebook will be broken down into a few steps:
* Loading and exploring the data
* Splitting the data into train/test sets
* Defining and training a LinearLearner, binary classifier
* Making improvements on the model
* Evaluating and comparing model test performance

### Making Improvements

A lot of this notebook will focus on making improvements, as discussed in [this SageMaker blog post](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/). Specifically, we'll address techniques for:

1. **Tuning a model's hyperparameters** and aiming for a specific metric, such as high recall or precision.
2. **Managing class imbalance**, which is when we have many more training examples in one class than another (in this case, many more valid transactions than fraudulent).

---

First, import the usual resources.

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

I'm storing my **SageMaker variables** in the next cell:
* sagemaker_session: The SageMaker session we'll use for training models.
* bucket: The name of the default S3 bucket that we'll use for data storage.
* role: The IAM role that defines our data and model permissions.

In [3]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


## Loading and Exploring the Data

Next, I am loading the data and unzipping the data in the file `creditcardfraud.zip`. This directory will hold one csv file of all the transaction data, `creditcard.csv`.

As in previous notebooks, it's important to look at the distribution of data since this will inform how we develop a fraud detection model. We'll want to know: How many data points we have to work with, the number and type of features, and finally, the distribution of data over the classes (valid or fraudulent).

In [4]:
# only have to run once
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
!unzip creditcardfraud

--2022-02-09 03:48:34--  https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.137.96
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.137.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip.2’

creditcardfraud.zip 100%[===================>]  65.95M  27.2MB/s    in 2.4s    

2022-02-09 03:48:37 (27.2 MB/s) - ‘creditcardfraud.zip.2’ saved [69155632/69155632]

Archive:  creditcardfraud.zip
replace creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
# read in the csv file
local_data = 'creditcard.csv'

# print out some data
transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)
print()
transaction_df.head()

Data shape (rows, cols):  (284807, 31)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### EXERCISE: Calculate the percentage of fraudulent data

Take a look at the distribution of this transaction data over the classes, valid and fraudulent. 

Complete the function `fraudulent_percentage`, below. Count up the number of data points in each class and calculate the *percentage* of the data points that are fraudulent.

In [6]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    # counts for all classes
    counts = transaction_df['Class'].value_counts()
    
    # get fraudulent and valid cnts
    fraud_cnts = counts[1]
    valid_cnts = counts[0]
    
    # calculate percentage of fraudulent data
    fraud_percentage = fraud_cnts/(fraud_cnts+valid_cnts)
    
    return fraud_percentage

Test out your code by calling your function and printing the result.

In [7]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])


Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


### EXERCISE: Split into train/test datasets

In this example, we'll want to evaluate the performance of a fraud classifier; training it on some training data and testing it on *test data* that it did not see during the training process. So, we'll need to split the data into separate training and test sets.

Complete the `train_test_split` function, below. This function should:
* Shuffle the transaction data, randomly
* Split it into two sets according to the parameter `train_frac`
* Get train/test features and labels
* Return the tuples: (train_features, train_labels), (test_features, test_labels)

In [8]:
# split into train/test
def split_data(transaction_df, test_size= 0.3, random_state=1):
   '''
   Shuffle the data and randomly split into train and test sets;
   separate the class labels (the column in transaction_df) from the features.
   :param df: Dataframe of all credit card transaction data
   :param train_frac: The decimal fraction of data that should be training data
   :param seed: Random seed for shuffling and reproducibility, default = 1
   :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
   '''
   
   train_df, test_df = train_test_split(transaction_df, test_size=test_size, random_state = random_state)
   train_features = train_df.drop(['Class'], axis=1)
   train_labels = train_df['Class']
   test_features = test_df.drop(['Class'], axis=1)
   test_labels = test_df['Class']

   return train_features, test_features, train_labels, test_labels


### Test Cell

In the cells below, I'm creating the train/test data and checking to see that result makes sense. The tests below test that the above function splits the data into the expected number of points and that the labels are indeed, class labels (0, 1).

In [9]:
# get train/test data
train_features, test_features, train_labels, test_labels = split_data(transaction_df, test_size=0.3, random_state=1)


In [10]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features.iloc[0, :])
print('Label: ', train_labels[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

Training data pts:  199364
Test data pts:  85443

First item: 
 Time      129124.000000
V1            -0.190075
V2             0.203323
V3            -0.996232
V4            -1.596985
V5             3.192579
V6             3.356935
V7             0.288299
V8             0.895003
V9            -0.330024
V10           -0.646900
V11           -0.282785
V12            0.023241
V13           -0.421092
V14            0.368247
V15           -0.158465
V16           -0.334756
V17           -0.337180
V18           -0.842032
V19           -0.033013
V20           -0.007496
V21           -0.170599
V22           -0.619724
V23            0.039651
V24            0.706805
V25           -0.160870
V26            0.274825
V27           -0.010541
V28            0.022199
Amount        14.370000
Name: 191125, dtype: float64
Label:  0

Tests passed!


---
# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

In this notebook, you'll define and train the SageMaker, built-in algorithm, [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html). 

A LinearLearner has two main applications:
1. For regression tasks in which a linear line is fit to some data points, and you want to produce a predicted output value given some data point (example: predicting house prices given square area).
2. For binary classification, in which a line is separating two classes of data and effectively outputs labels; either 1 for data that falls above the line or 0 for points that fall on or below the line.

<img src='notebook_ims/linear_separator.png' width=40% />

In this case, we'll be using it for case 2, and we'll train it to separate data into our two classes: valid or fraudulent. 

### EXERCISE: Create a LinearLearner Estimator

You've had some practice instantiating built-in models in SageMaker. All estimators require some constructor arguments to be passed in. See if you can complete this task, instantiating a LinearLearner estimator, using only the [LinearLearner documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html) as a resource. This takes in a lot of arguments, but not all are required. My suggestion is to start with a simple model, utilizing default values where applicable. Later, we will discuss some specific hyperparameters and their use cases.

#### Instance Types

It is suggested that you use instances that are available in the free tier of usage: `'ml.c4.xlarge'` for training and `'ml.t2.medium'` for deployment.

In [11]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
prefix = 'creditcard'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15)


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### EXERCISE: Convert data into a RecordSet format

Next, prepare the data for a built-in model by converting the train features and labels into numpy array's of float values. Then you can use the [record_set function](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set) to format the data as a RecordSet and prepare it for training!

In [12]:
# convert features/labels to numpy
train_x_np = train_features.values.astype('float32')
train_y_np = train_labels.values.astype('float32')

# create RecordSet
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

### EXERCISE: Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`, passing in the formatted training data.

In [13]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2022-02-09 03:48:51 Starting - Starting the training job...
2022-02-09 03:49:14 Starting - Launching requested ML instancesProfilerReport-1644378531: InProgress
.........
2022-02-09 03:50:35 Starting - Preparing the instances for training......
2022-02-09 03:51:35 Downloading - Downloading input data......
2022-02-09 03:52:41 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script

2022-02-09 03:53:15 Training - Training image download completed. Training in progress.[02/09/2022 03:53:09 INFO 139910443296576] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_metho

[2022-02-09 03:53:23.264] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 6469, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644378803.2649584, "EndTime": 1644378803.2650461, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.016883339575187645, "count": 1, "min": 0.016883339575187645, "max": 0.016883339575187645}}}
#metrics {"StartTime": 1644378803.2651317, "EndTime": 1644378803.2651513, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014893327607581363, "count": 1, "min": 0.014893327607581363, "max": 0.014893327607581363}}}
#metrics {"StartTime": 1644378803.265206, "EndTime": 1644378803.2652223, "Dimensions": {"Algorithm": "Linear L

[2022-02-09 03:53:30.006] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6730, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644378810.0067787, "EndTime": 1644378810.0068583, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.010249469512671083, "count": 1, "min": 0.010249469512671083, "max": 0.010249469512671083}}}
#metrics {"StartTime": 1644378810.0069573, "EndTime": 1644378810.0069773, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008671593309047832, "count": 1, "min": 0.008671593309047832, "max": 0.008671593309047832}}}
#metrics {"StartTime": 1644378810.0070307, "EndTime": 1644378810.0070426, "Dimensions": {"Algorithm": "Linear 

[2022-02-09 03:53:43.104] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6419, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644378823.1045194, "EndTime": 1644378823.1046028, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006847595499987578, "count": 1, "min": 0.006847595499987578, "max": 0.006847595499987578}}}
#metrics {"StartTime": 1644378823.1046872, "EndTime": 1644378823.1047058, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005652893664249823, "count": 1, "min": 0.005652893664249823, "max": 0.005652893664249823}}}
#metrics {"StartTime": 1644378823.1047568, "EndTime": 1644378823.1047761, "Dimensions": {"Algorithm": "Linear

...

[2022-02-09 03:54:09.414] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 19, "duration": 6866, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644378849.4144292, "EndTime": 1644378849.4145138, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005324880617946836, "count": 1, "min": 0.005324880617946836, "max": 0.005324880617946836}}}
#metrics {"StartTime": 1644378849.414592, "EndTime": 1644378849.4146106, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004463636301869723, "count": 1, "min": 0.004463636301869723, "max": 0.004463636301869723}}}
#metrics {"StartTime": 1644378849.4146643, "EndTime": 1644378849.4146802, "Dimensions": {"Algorithm": "Linear 

[2022-02-09 03:54:22.246] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6384, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644378862.2467556, "EndTime": 1644378862.2469013, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0050622782317837275, "count": 1, "min": 0.0050622782317837275, "max": 0.0050622782317837275}}}
#metrics {"StartTime": 1644378862.247024, "EndTime": 1644378862.2470484, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00428980573787162, "count": 1, "min": 0.00428980573787162, "max": 0.00428980573787162}}}
#metrics {"StartTime": 1644378862.247114, "EndTime": 1644378862.2471309, "Dimensions": {"Algorithm": "Linear

[2022-02-09 03:54:35.024] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6594, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644378875.0244696, "EndTime": 1644378875.0245545, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004901528771798215, "count": 1, "min": 0.004901528771798215, "max": 0.004901528771798215}}}
#metrics {"StartTime": 1644378875.0246408, "EndTime": 1644378875.024659, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004192210530515891, "count": 1, "min": 0.004192210530515891, "max": 0.004192210530515891}}}
#metrics {"StartTime": 1644378875.0247147, "EndTime": 1644378875.0247326, "Dimensions": {"Algorithm": "Linea


2022-02-09 03:54:56 Uploading - Uploading generated training model[2022-02-09 03:54:41.707] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6670, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644378881.707277, "EndTime": 1644378881.7073648, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00484367591711744, "count": 1, "min": 0.00484367591711744, "max": 0.00484367591711744}}}
#metrics {"StartTime": 1644378881.7075102, "EndTime": 1644378881.7075326, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0041608323353019795, "count": 1, "min": 0.0041608323353019795, "max": 0.0041608323353019795}}}
#metrics {"StartTime": 1644378881.7075908


2022-02-09 03:55:16 Completed - Training job completed
ProfilerReport-1644378531: NoIssuesFound
Training seconds: 215
Billable seconds: 215
CPU times: user 895 ms, sys: 55.1 ms, total: 950 ms
Wall time: 6min 47s


### EXERCISE: Deploy the trained model

Deploy your model to create a predictor. We'll use this to make predictions on our test data and evaluate the model.

In [14]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------!CPU times: user 210 ms, sys: 17.1 ms, total: 227 ms
Wall time: 6min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to the test data.

According to the deployed [predictor documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor), this predictor expects an `ndarray` of input features and returns a list of Records.
> "The prediction is stored in the "predicted_label" key of the `Record.label` field."

Let's first test our model on just one test point, to see the resulting list.

In [15]:
# test one prediction
test_x_np = test_features.values.astype('float32')
result = linear_predictor.predict(test_x_np[0])

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.0002751190622802824
    }
  }
}
]


### Helper function for evaluation


The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [16]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `test_features` and `test_labels` from previously-run cells.

In [17]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 85282   26
1                    36   99

Recall:     0.733
Precision:  0.792
Accuracy:   0.999



We can see that this model gets a very high accuracy of 99.9% ! But it still misclassifies about 30 (each) of our valid and fraudulent cases, which results in much lower values for recall and precision.

Next, let's delete this endpoint and discuss ways to improve this model.

## Delete the Endpoint

I've added a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [18]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [19]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2022-02-09-03-55-38-985


---

# Model Improvements

The default LinearLearner got a high accuracy, but still classified fraudulent and valid data points incorrectly. Specifically classifying more than 30 points as false negatives (incorrectly labeled, fraudulent transactions), and a little over 30 points as false positives (incorrectly labeled, valid transactions). Let's think about what, during training, could cause this behavior and what we could improve.

**1. Model optimization**
* If we imagine that we are designing this model for use in a bank application, we know that users do *not* want any valid transactions to be categorized as fraudulent. That is, we want to have as few **false positives** (0s classified as 1s) as possible. 
* On the other hand, if our bank manager asks for an application that will catch almost *all* cases of fraud, even if it means a higher number of false positives, then we'd want as few **false negatives** as possible.
* To train according to specific product demands and goals, we do not want to optimize for accuracy only. Instead, we want to optimize for a metric that can help us decrease the number of false positives or negatives. 

<img src='notebook_ims/precision_recall.png' width=40% />
     
In this notebook, we'll look at different cases for tuning a model and make an optimization decision, accordingly.

**2. Imbalanced training data**
* At the start of this notebook, we saw that only about 0.17% of the data was labeled as fraudulent. So, even if a model labels **all** of our data as valid, it will still have a high accuracy. 
* This may result in some overfitting towards valid data, which accounts for some **false negatives**; cases in which fraudulent data (1) is incorrectly characterized as valid (0).

So, let's address these issues in order; first, tuning our model and optimizing for a specific metric during training, and second, accounting for class imbalance in the training set. 


## Improvement: Model Tuning

Optimizing according to a specific metric is called **model tuning**, and SageMaker provides a number of ways to automatically tune a model.


### Create a LinearLearner and tune for higher precision 

**Scenario:**
* A bank has asked you to build a model that detects cases of fraud with an accuracy of about 85%. 

In this case, we want to build a model that has as many true positives and as few false negatives, as possible. This corresponds to a model with a high **recall**: true positives / (true positives + false negatives). 

To aim for a specific metric, LinearLearner offers the hyperparameter `binary_classifier_model_selection_criteria`, which is the model evaluation criteria for the training dataset. A reference to this parameter is in [LinearLearner's documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner). We'll also have to further specify the exact value we want to aim for; read more about the details of the parameters, [here](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html).

I will assume that performance on a training set will be within about 5% of the performance on a test set. So, for a recall of about 85%, I'll aim for a bit higher, 90%.

In [20]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train the tuned estimator

Fit the new, tuned estimator on the formatted training data.

In [21]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2022-02-09 04:02:22 Starting - Starting the training job...
2022-02-09 04:02:24 Starting - Launching requested ML instancesProfilerReport-1644379341: InProgress
......
2022-02-09 04:03:52 Starting - Preparing the instances for training............
2022-02-09 04:05:47 Downloading - Downloading input data...
2022-02-09 04:06:21 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[02/09/2022 04:06:27 INFO 140186561488704] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'ini

[2022-02-09 04:06:41.381] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7051, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644379601.3818371, "EndTime": 1644379601.3819451, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.016883339575187645, "count": 1, "min": 0.016883339575187645, "max": 0.016883339575187645}}}
#metrics {"StartTime": 1644379601.3820364, "EndTime": 1644379601.3820572, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014893327607581363, "count": 1, "min": 0.014893327607581363, "max": 0.014893327607581363}}}
#metrics {"StartTime": 1644379601.3821077, "EndTime": 1644379601.3821232, "Dimensions": {"Algorithm": "Linear 

[2022-02-09 04:06:48.487] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 7094, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644379608.488012, "EndTime": 1644379608.4881113, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.010249469512671083, "count": 1, "min": 0.010249469512671083, "max": 0.010249469512671083}}}
#metrics {"StartTime": 1644379608.4882302, "EndTime": 1644379608.4882548, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008671593309047832, "count": 1, "min": 0.008671593309047832, "max": 0.008671593309047832}}}
#metrics {"StartTime": 1644379608.4883134, "EndTime": 1644379608.4883308, "Dimensions": {"Algorithm": "Linear L

[2022-02-09 04:07:02.566] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 7036, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644379622.5668192, "EndTime": 1644379622.5669265, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006847595499987578, "count": 1, "min": 0.006847595499987578, "max": 0.006847595499987578}}}
#metrics {"StartTime": 1644379622.5670485, "EndTime": 1644379622.5670729, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005652893664249823, "count": 1, "min": 0.005652893664249823, "max": 0.005652893664249823}}}
#metrics {"StartTime": 1644379622.5671425, "EndTime": 1644379622.5671594, "Dimensions": {"Algorithm": "Linear

...

[2022-02-09 04:07:35.401] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 7043, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644379655.4015248, "EndTime": 1644379655.4016268, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005176234991706196, "count": 1, "min": 0.005176234991706196, "max": 0.005176234991706196}}}
#metrics {"StartTime": 1644379655.4017441, "EndTime": 1644379655.401768, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004362526129538091, "count": 1, "min": 0.004362526129538091, "max": 0.004362526129538091}}}
#metrics {"StartTime": 1644379655.4018266, "EndTime": 1644379655.4018447, "Dimensions": {"Algorithm": "Linear 

[2022-02-09 04:07:41.925] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6512, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644379661.9256525, "EndTime": 1644379661.925749, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0050622782317837275, "count": 1, "min": 0.0050622782317837275, "max": 0.0050622782317837275}}}
#metrics {"StartTime": 1644379661.9258523, "EndTime": 1644379661.9258735, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00428980573787162, "count": 1, "min": 0.00428980573787162, "max": 0.00428980573787162}}}
#metrics {"StartTime": 1644379661.925925, "EndTime": 1644379661.9259431, "Dimensions": {"Algorithm": "Linear

[2022-02-09 04:07:55.204] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6416, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644379675.204788, "EndTime": 1644379675.2048852, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004901528771798215, "count": 1, "min": 0.004901528771798215, "max": 0.004901528771798215}}}
#metrics {"StartTime": 1644379675.204988, "EndTime": 1644379675.2050095, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004192210530515891, "count": 1, "min": 0.004192210530515891, "max": 0.004192210530515891}}}
#metrics {"StartTime": 1644379675.2050657, "EndTime": 1644379675.2050853, "Dimensions": {"Algorithm": "Linear

[2022-02-09 04:08:01.277] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6058, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644379681.2773175, "EndTime": 1644379681.2774215, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00484367591711744, "count": 1, "min": 0.00484367591711744, "max": 0.00484367591711744}}}
#metrics {"StartTime": 1644379681.2775328, "EndTime": 1644379681.2775567, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0041608323353019795, "count": 1, "min": 0.0041608323353019795, "max": 0.0041608323353019795}}}
#metrics {"StartTime": 1644379681.2776048, "EndTime": 1644379681.277616, "Dimensions": {"Algorithm": "Linea


2022-02-09 04:08:22 Uploading - Uploading generated training model
2022-02-09 04:08:22 Completed - Training job completed
Training seconds: 155
Billable seconds: 155
CPU times: user 854 ms, sys: 40.4 ms, total: 895 ms
Wall time: 6min 17s


### Deploy and evaluate the tuned estimator

Deploy the tuned predictor and evaluate it.

We hypothesized that a tuned model, optimized for a higher recall, would have fewer false negatives (fraudulent transactions incorrectly labeled as valid); did the number of false negatives get reduced after tuning the model?

In [22]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------!CPU times: user 217 ms, sys: 8.31 ms, total: 225 ms
Wall time: 6min 32s


In [23]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 84317  991
1                    20  115

Recall:     0.852
Precision:  0.104
Accuracy:   0.988



## Delete the endpoint 

As always, when you're done evaluating a model, you should delete the endpoint. Below, I'm using the `delete_endpoint` helper function I defined earlier.

In [24]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2022-02-09-04-08-38-831


---
## Improvement: Managing Class Imbalance

We have a model that is tuned to get a higher recall, which aims to reduce the number of false negatives. Earlier, we discussed how class imbalance may actually bias our model towards predicting that all transactions are valid, resulting in higher false negatives and true negatives. It stands to reason that this model could be further improved if we account for this imbalance.

To account for class imbalance during training of a binary classifier, LinearLearner offers the hyperparameter, `positive_example_weight_mult`, which is the weight assigned to positive (1, fraudulent) examples when training a binary classifier. The weight of negative examples (0, valid) is fixed at 1. 

### EXERCISE: Create a LinearLearner with a `positive_example_weight_mult` parameter

In **addition** to tuning a model for higher recall (you may use `linear_recall` as a starting point), you should *add* a parameter that helps account for class imbalance. From the [hyperparameter documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html) on `positive_example_weight_mult`, it reads:
> "If you want the algorithm to choose a weight so that errors in classifying negative vs. positive examples have equal impact on training loss, specify `balanced`."

You could also put in a specific float value, in which case you'd want to weight positive examples more heavily than negative examples, since there are fewer of them.

In [25]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                                target_recall=0.9,
                                positive_example_weight_mult='balanced')


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### EXERCISE: Train the balanced estimator

Fit the new, balanced estimator on the formatted training data.

In [26]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2022-02-09 04:15:22 Starting - Starting the training job...
2022-02-09 04:15:49 Starting - Launching requested ML instancesProfilerReport-1644380122: InProgress
.........
2022-02-09 04:17:18 Starting - Preparing the instances for training.........
2022-02-09 04:18:53 Downloading - Downloading input data...
2022-02-09 04:19:19 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[02/09/2022 04:19:24 INFO 140005477537600] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'ini

[2022-02-09 04:19:45.435] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6822, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644380385.435126, "EndTime": 1644380385.4352162, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3391149269755761, "count": 1, "min": 0.3391149269755761, "max": 0.3391149269755761}}}
#metrics {"StartTime": 1644380385.4352994, "EndTime": 1644380385.4353175, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3195404147047493, "count": 1, "min": 0.3195404147047493, "max": 0.3195404147047493}}}
#metrics {"StartTime": 1644380385.4353716, "EndTime": 1644380385.4353893, "Dimensions": {"Algorithm": "Li

[2022-02-09 04:19:52.284] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6837, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644380392.2848854, "EndTime": 1644380392.2849603, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.32226599174767884, "count": 1, "min": 0.32226599174767884, "max": 0.32226599174767884}}}
#metrics {"StartTime": 1644380392.2850745, "EndTime": 1644380392.28509, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3074733453491824, "count": 1, "min": 0.3074733453491824, "max": 0.3074733453491824}}}
#metrics {"StartTime": 1644380392.2851284, "EndTime": 1644380392.2851422, "Dimensions": {"Algorithm": "

[2022-02-09 04:20:06.992] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 7535, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644380406.9927378, "EndTime": 1644380406.9928143, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.30667649917985923, "count": 1, "min": 0.30667649917985923, "max": 0.30667649917985923}}}
#metrics {"StartTime": 1644380406.992919, "EndTime": 1644380406.992936, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29720698666213147, "count": 1, "min": 0.29720698666213147, "max": 0.29720698666213147}}}
#metrics {"StartTime": 1644380406.9929795, "EndTime": 1644380406.9929905, "Dimensions": {"Algorithm

...

[2022-02-09 04:20:40.965] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 7028, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644380440.965584, "EndTime": 1644380440.965671, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2939692262045702, "count": 1, "min": 0.2939692262045702, "max": 0.2939692262045702}}}
#metrics {"StartTime": 1644380440.9657657, "EndTime": 1644380440.9657862, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28833337498190414, "count": 1, "min": 0.28833337498190414, "max": 0.28833337498190414}}}
#metrics {"StartTime": 1644380440.9658418, "EndTime": 1644380440.9658573, "Dimensions": {"Algorithm"

[2022-02-09 04:20:48.243] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 7265, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644380448.2438297, "EndTime": 1644380448.2439194, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2928580866674682, "count": 1, "min": 0.2928580866674682, "max": 0.2928580866674682}}}
#metrics {"StartTime": 1644380448.244017, "EndTime": 1644380448.2440386, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2874274079979365, "count": 1, "min": 0.2874274079979365, "max": 0.2874274079979365}}}
#metrics {"StartTime": 1644380448.2440975, "EndTime": 1644380448.2441154, "Dimensions": {"Algorithm": 

[2022-02-09 04:21:01.003] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6343, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644380461.0031254, "EndTime": 1644380461.0032108, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29126332237972086, "count": 1, "min": 0.29126332237972086, "max": 0.29126332237972086}}}
#metrics {"StartTime": 1644380461.0032926, "EndTime": 1644380461.0033116, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2859753795221223, "count": 1, "min": 0.2859753795221223, "max": 0.2859753795221223}}}
#metrics {"StartTime": 1644380461.0033648, "EndTime": 1644380461.0033803, "Dimensions": {"Algorith

[2022-02-09 04:21:08.355] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 7338, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644380468.355374, "EndTime": 1644380468.355433, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29068308453583835, "count": 1, "min": 0.29068308453583835, "max": 0.29068308453583835}}}
#metrics {"StartTime": 1644380468.3555102, "EndTime": 1644380468.355528, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28538028095954626, "count": 1, "min": 0.28538028095954626, "max": 0.28538028095954626}}}
#metrics {"StartTime": 1644380468.355577, "EndTime": 1644380468.3555918, "Dimensions": {"Algorithm


2022-02-09 04:21:31 Uploading - Uploading generated training model
2022-02-09 04:21:31 Completed - Training job completed
ProfilerReport-1644380122: NoIssuesFound
Training seconds: 147
Billable seconds: 147
CPU times: user 935 ms, sys: 43.1 ms, total: 978 ms
Wall time: 6min 25s


### EXERCISE: Deploy and evaluate the balanced estimator

Deploy the balanced predictor and evaluate it. Do the results match with your expectations?

In [27]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------!CPU times: user 190 ms, sys: 9.18 ms, total: 200 ms
Wall time: 5min 32s


In [28]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 84629  679
1                    18  117

Recall:     0.867
Precision:  0.147
Accuracy:   0.992



## Delete the endpoint 

When you're done evaluating a model, you should delete the endpoint.

In [29]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2022-02-09-04-21-47-330


A note on metric variability: 

The above model is tuned for the best possible precision with recall fixed at about 90%. The recall is fixed at 90% during training, but may vary when we apply our trained model to a test set of data.

---
## Model Design

Now that you've seen how to tune and balance a LinearLearner. Create, train and deploy your own model. This exercise is meant to be more open-ended, so that you get practice with the steps involved in designing a model and deploying it.

### EXERCISE: Train and deploy a LinearLearner with appropriate hyperparameters, according to the given scenario

**Scenario:**
* A bank has asked you to build a model that optimizes for a good user experience; users should only ever have up to about 15% of their valid transactions flagged as fraudulent.

This requires that you make a design decision: Given the above scenario, what metric (and value) should you aim for during training?

You may assume that performance on a training set will be within about 5-10% of the performance on a test set. For example, if you get 80% on a training set, you can assume that you'll get between about 70-90% accuracy on a test set.

Your final model should account for class imbalance and be appropriately tuned. 

If we're allowed about 15/100 incorrectly classified valid transactions (false positives), then I can calculate an approximate value for the precision that I want as: 85/(85+15) = 85%. I'll aim for about 5% higher during training to ensure that I get closer to 80-85% precision on the test data.

In [30]:
%%time
# instantiate and train a LinearLearner

# include params for tuning for higher precision
# *and* account for class imbalance in training data
linear_precision = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='recall_at_target_precision',
                                target_precision=0.9,
                                positive_example_weight_mult='balanced')


# train the estimator on formatted training data
linear_precision.fit(formatted_train_data)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2022-02-09 04:27:29 Starting - Starting the training job...
2022-02-09 04:27:53 Starting - Launching requested ML instancesProfilerReport-1644380849: InProgress
.........
2022-02-09 04:29:14 Starting - Preparing the instances for training.........
2022-02-09 04:30:59 Downloading - Downloading input data
2022-02-09 04:30:59 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[02/09/2022 04:31:05 INFO 139798755358528] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_b

[2022-02-09 04:31:19.583] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 6847, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644381079.5837393, "EndTime": 1644381079.583816, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3762333395109704, "count": 1, "min": 0.3762333395109704, "max": 0.3762333395109704}}}
#metrics {"StartTime": 1644381079.5838947, "EndTime": 1644381079.5839133, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3452077499849712, "count": 1, "min": 0.3452077499849712, "max": 0.3452077499849712}}}
#metrics {"StartTime": 1644381079.5839648, "EndTime": 1644381079.5839822, "Dimensions": {"Algorithm": "Li

[2022-02-09 04:31:26.641] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 7044, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644381086.6412818, "EndTime": 1644381086.6413622, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3391149269755761, "count": 1, "min": 0.3391149269755761, "max": 0.3391149269755761}}}
#metrics {"StartTime": 1644381086.641453, "EndTime": 1644381086.641468, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3195404147047493, "count": 1, "min": 0.3195404147047493, "max": 0.3195404147047493}}}
#metrics {"StartTime": 1644381086.6415021, "EndTime": 1644381086.6415107, "Dimensions": {"Algorithm": "Lin

[2022-02-09 04:31:39.971] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6492, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644381099.9711516, "EndTime": 1644381099.9712315, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3127615477978884, "count": 1, "min": 0.3127615477978884, "max": 0.3127615477978884}}}
#metrics {"StartTime": 1644381099.9713144, "EndTime": 1644381099.9713314, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.30119617619346734, "count": 1, "min": 0.30119617619346734, "max": 0.30119617619346734}}}
#metrics {"StartTime": 1644381099.9714146, "EndTime": 1644381099.971433, "Dimensions": {"Algorithm":

...

[2022-02-09 04:32:07.963] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 19, "duration": 7264, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644381127.9635284, "EndTime": 1644381127.9636157, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2971283764096361, "count": 1, "min": 0.2971283764096361, "max": 0.2971283764096361}}}
#metrics {"StartTime": 1644381127.963699, "EndTime": 1644381127.9637175, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2907087844005182, "count": 1, "min": 0.2907087844005182, "max": 0.2907087844005182}}}
#metrics {"StartTime": 1644381127.9637704, "EndTime": 1644381127.963786, "Dimensions": {"Algorithm": "Li

[2022-02-09 04:32:21.078] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6564, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644381141.0785093, "EndTime": 1644381141.0785902, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2939692262045702, "count": 1, "min": 0.2939692262045702, "max": 0.2939692262045702}}}
#metrics {"StartTime": 1644381141.07866, "EndTime": 1644381141.078677, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28833337498190414, "count": 1, "min": 0.28833337498190414, "max": 0.28833337498190414}}}
#metrics {"StartTime": 1644381141.078728, "EndTime": 1644381141.078743, "Dimensions": {"Algorithm": "

[2022-02-09 04:32:27.868] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 6777, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644381147.8687477, "EndTime": 1644381147.8688247, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2928580866674682, "count": 1, "min": 0.2928580866674682, "max": 0.2928580866674682}}}
#metrics {"StartTime": 1644381147.8689163, "EndTime": 1644381147.8689368, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2874274079979365, "count": 1, "min": 0.2874274079979365, "max": 0.2874274079979365}}}
#metrics {"StartTime": 1644381147.8689966, "EndTime": 1644381147.8690135, "Dimensions": {"Algorithm":

[2022-02-09 04:32:41.840] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6951, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1644381161.8402221, "EndTime": 1644381161.8402987, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29126332237972086, "count": 1, "min": 0.29126332237972086, "max": 0.29126332237972086}}}
#metrics {"StartTime": 1644381161.840382, "EndTime": 1644381161.8403974, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2859753795221223, "count": 1, "min": 0.2859753795221223, "max": 0.2859753795221223}}}
#metrics {"StartTime": 1644381161.8404384, "EndTime": 1644381161.840448, "Dimensions": {"Algorithm"


2022-02-09 04:33:00 Uploading - Uploading generated training model
2022-02-09 04:33:00 Completed - Training job completed
Training seconds: 149
Billable seconds: 149
CPU times: user 803 ms, sys: 38.5 ms, total: 841 ms
Wall time: 5min 45s


This model trains for a fixed precision of 90%, and, under that constraint, tries to get as high a recall as possible.

In [31]:
%%time 
# deploy and evaluate a predictor
precision_predictor = linear_precision.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


--------!CPU times: user 152 ms, sys: 4.92 ms, total: 157 ms
Wall time: 4min 1s


In [32]:
print('Metrics for tuned (precision), LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(precision_predictor, 
                   test_features.values.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (precision), LinearLearner.

prediction (col)  0.0    1.0
actual (row)                
0                 929  84379
1                   0    135

Recall:     1.000
Precision:  0.002
Accuracy:   0.012



In [33]:
## IMPORTANT
# delete the predictor endpoint 
delete_endpoint(precision_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2022-02-09-04-33-15-401


## Final Cleanup!

* Double check that you have deleted all your endpoints.
* I'd also suggest manually deleting your S3 bucket, models, and endpoint configurations directly from your AWS console.

You can find thorough cleanup instructions, [in the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).

---
# Conclusion

In this notebook, you saw how to train and deploy a LinearLearner in SageMaker. This model is well-suited for a binary classification task that involves specific design decisions and managing class imbalance in the training set.

Following the steps of a machine learning workflow, you loaded in some credit card transaction data, explored that data and prepared it for model training. Then trained, deployed, and evaluated several models, according to different design considerations!